In [9]:
# Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [11]:
# Set your dataset directory
DATASET_DIR = 'data/ugv_dataset_v7/train'  # Update this to your dataset path
images_dir = os.path.join(DATASET_DIR, 'images')
labels_dir = os.path.join(DATASET_DIR, 'labels')

In [12]:
# List all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
image_files.sort()  # Sort to maintain order

# Create an output widget
output = widgets.Output()

# Define a color map for different classes
color_map = {
    0: (255, 0, 0),    # Red for class 0
    1: (0, 255, 0),    # Green for class 1
    2: (0, 0, 255),    # Blue for class 2
    3: (255, 255, 0),  # Cyan for class 3
    4: (255, 0, 255),  # Magenta for class 4
    5: (0, 255, 255)   # Yellow for class 5
    # Add more colors as needed for additional classes
}

# Function to display an image with annotations
def display_image(index):
    with output:
        output.clear_output(True)

        img_path = os.path.join(images_dir, image_files[index])
        label_path = os.path.join(labels_dir, image_files[index].replace('.jpg', '.txt').replace('.png', '.txt'))
        
        # Read image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for displaying

        # Read labels
        boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    boxes.append((class_id, x_center, y_center, width, height))
        
        # Calculate thickness based on image resolution
        height, width, _ = image.shape
        thickness = max(2, int(height * 0.005))  # Set thickness to 0.5% of image height (minimum of 1)

        # Draw boxes on the image
        for box in boxes:
            class_id, x_center, y_center, width, height = box
            x1 = int((x_center - width / 2) * image.shape[1])
            y1 = int((y_center - height / 2) * image.shape[0])
            x2 = int((x_center + width / 2) * image.shape[1])
            y2 = int((y_center + height / 2) * image.shape[0])
            color = color_map.get(int(class_id), (255, 255, 255))  # Default to white if class not in map
            cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
            cv2.putText(image, str(int(class_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Image: {image_files[index]}")
        plt.show()

# Create a slider to browse through images
image_slider = widgets.IntSlider(value=0, min=0, max=len(image_files)-1, step=1, description='Image Index')

# Set up an interactive output
interactive_display = widgets.interactive(display_image, index=image_slider)

# Display the slider and output area
display(interactive_display, output)


interactive(children=(IntSlider(value=0, description='Image Index', max=2142), Output()), _dom_classes=('widge…

Output()